# 데이터 불러오기

In [9]:
import pandas as pd
X_train = pd.read_csv('stellar_X_train.csv')
y_train = pd.read_csv('stellar_y_train.csv')
X_test = pd.read_csv('stellar_X_test.csv')

# 데이터 확인

In [ ]:
print(X_train.info())

In [4]:
print(X_train.describe())

              alpha         delta             u             g             r  \
count  70000.000000  70000.000000  70000.000000  70000.000000  70000.000000   
mean     177.535809     24.107885     21.938806     20.486718     19.644011   
std       96.483093     19.665057     37.942814     37.925201      1.852053   
min        0.005528    -18.785328  -9999.000000  -9999.000000      9.822070   
25%      127.385304      5.115922     20.349987     18.957325     18.136183   
50%      180.706865     23.579507     22.183860     21.095445     20.122805   
75%      233.942154     39.871336     23.684145     22.121073     21.039012   
max      359.999615     82.947622     32.781390     31.602240     29.571860   

                  i             z      redshift        run_ID  rerun_ID  \
count  70000.000000  70000.000000  70000.000000  70000.000000   70000.0   
mean      19.082969     18.624590      0.575246   4478.649443     301.0   
std        1.755997     37.904705      0.729921   1965.326847  

# 이상치 제거

In [10]:
y_train = y_train[X_train['u']!=-9999]
X_train = X_train[X_train['u']!=-9999]

y_train = y_train[X_train['g']!=-9999]
X_train = X_train[X_train['g']!=-9999]

y_train = y_train[X_train['z']!=-9999]
X_train = X_train[X_train['z']!=-9999]

In [11]:
X_train.describe()

,alpha,delta,u,g,r,i,z,redshift,run_ID,rerun_ID,cam_col,field_ID,plate
count,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.0,69999.000000,69999.000000,69999.000000
mean,177.535145,24.108238,22.081964,20.629856,19.644032,19.082984,18.767701,0.575254,4478.702681,301.0,3.510522,186.040701,5127.878813
std,96.483622,19.664976,2.248554,2.033576,1.852057,1.756005,1.764734,0.729923,1965.290408,0.0,1.585204,148.656411,2947.466071
min,0.005528,-18.785328,10.996230,10.498200,9.822070,9.469903,9.612333,-0.009971,109.000000,301.0,1.000000,11.000000,266.000000
25%,127.384192,5.117238,20.350000,18.957435,18.136075,17.733685,17.463885,0.052608,3185.000000,301.0,2.000000,82.000000,2525.500000
50%,180.705886,23.580630,22.183900,21.095450,20.122810,19.399590,19.001020,0.422848,4188.000000,301.0,4.000000,146.000000,4978.000000
75%,233.942330,39.872079,23.684160,22.121075,21.039035,20.391350,19.917595,0.700399,5326.000000,301.0,5.000000,241.000000,7389.000000
max,359.999615,82.947622,32.781390,31.602240,29.571860,32.141470,28.790550,7.011245,8162.000000,301.0,6.000000,989.000000,12547.000000


# 스케일링

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()
X_train[['alpha','delta']] = scaler.fit_transform(X_train[['alpha','delta']])
# train데이터로 fit했으면 test데이터는 transform만 실행
X_test[['alpha','delta']] = scaler.transform(X_test[['alpha','delta']])

In [ ]:
print(X_train.describe())

# 데이터 분할

In [16]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(X_train, y_train)

In [17]:
print(x_tr.shape, x_val.shape, y_tr.shape, y_val.shape)

(52499, 13) (17500, 13) (52499, 1) (17500, 1)


# 모델링

In [18]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_tr, y_tr)
pred = rf.predict(x_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
# xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(x_tr, y_tr)
pred2 = xgb.predict(x_val)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 평가

In [21]:
from sklearn.metrics import roc_auc_score
score1 = roc_auc_score(y_val, pred)
score2 = roc_auc_score(y_val, pred2)

In [22]:
print(score1, score2) # score1이 더 높으므로 랜덤포레스트로 결정

0.9768273906490794 0.974133108158421


In [24]:
print(rf.predict_proba(X_test)[:,1])

[0.97 0.   1.   ... 0.05 0.04 0.98]


# 제출

In [26]:
result = rf.predict_proba(X_test)[:,1]
# CSV를 index, galaxy 로 저장
pd.DataFrame({'index':X_test.index, 'galaxy':result}).to_csv('1234.csv', index=False)

In [27]:
# 확인
pd.read_csv('1234.csv')

,index,galaxy
0,0,0.97
1,1,0.00
2,2,1.00
3,3,1.00
4,4,0.03
...,...,...
29995,29995,0.98
29996,29996,0.01
29997,29997,0.05
29998,29998,0.04
